<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/melaflefon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wav2Lip Demo Colab

https://bhaasha.iiit.ac.in/lipsync

In [18]:
#@title <- Press play, then press "Browse" below and upload image(s) or video(s)
%tensorflow_version 1.x
%cd /content
!git clone https://github.com/Rudrabha/Wav2Lip.git
import os
if not os.path.exists('/content/Wav2Lip/checkpoints/wav2lip_gan.pth'):
  !gdown https://drive.google.com/uc?id=1dwHujX7RVNCvdR1RR93z0FS2T2yzqup9 -O /content/Wav2Lip/checkpoints/wav2lip_gan.pth
!wget --no-check-certificate -nc https://drive.google.com/uc?id=18yqnoefpT32lC-f1pqvMXtVLNYbyYEDk -O /content/melaflefon.mp3
!wget --no-check-certificate -nc https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"
from google.colab import files
%cd /content/sample_data
images = files.upload()
%cd /content/Wav2Lip
outputs = []
for im in images:
  infile = '/content/sample_data/'+im
  outfile = '/content/'+im.rsplit('.')[0]+'_out.mp4'
  !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '$infile' --audio "/content/melaflefon.mp3" --pads 0 20 0 0 --outfile \"'$outfile'\" 2>&1 | tee /content/out.txt
  bad = !cat /content/out.txt | grep 'Face not detected'
  if bad:
    import cv2
    print('\nFace not detected - will use whole frame')
    if infile.split('.')[1] in ['jpg', 'png', 'jpeg']:
      frame = cv2.imread(infile)
    else:
      video_stream = cv2.VideoCapture(infile)
      still_reading, frame = video_stream.read()
    h,w = frame.shape[:2]
    !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '$infile' --audio "/content/melaflefon.mp3" --box 0 $h 0 $w --outfile \"'$outfile'\"
  outputs.append(outfile)
  os.remove(infile)
for file in outputs:
  try:
    files.download(file)
  except:
    pass

/content
fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
File ‘/content/melaflefon.mp3’ already there; not retrieving.
File ‘/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth’ already there; not retrieving.
/content/sample_data


Saving smile.mp4 to smile.mp4
/content/Wav2Lip
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>